<a href="https://colab.research.google.com/github/joshikavita01/Interview/blob/main/Assignment_3_Ques(1_c).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Question 1: Image Classification using Handcrafted and Deep Features**

**(c) Use AlexNet pre-trained on Imagenet to extract the 4096-d feature vector)**

In [82]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

In [83]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [84]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [85]:
training_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(training_data, batch_size=4, shuffle=True, num_workers=2)
#test data
testing_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testing_data, batch_size=1, shuffle=False, num_workers=2)
#Class labels
classes = ('Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

Files already downloaded and verified
Files already downloaded and verified


In [86]:
import torch
import torch.nn as nn
from torchvision import models
m = models.alexnet(pretrained=True)
# remove last fully-connected layer
new_classifier = nn.Sequential(*list(m.classifier.children())[:-1])
m.classifier = new_classifier

In [87]:
m.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [88]:
m.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [89]:
@torch.no_grad()
def get_predictions(model, loader):
  _predictions = torch.tensor([])
  _predictions =_predictions.to(device)
  for batch in loader:
    images, labels = batch
    images, labels = images.to(device), labels.to(device)

    predictions = model(images)
    _predictions = torch.cat((_predictions, predictions),dim=0)

  return _predictions


pred_output = get_predictions(m,test_loader)


In [90]:
class Network(nn.Module):
  def __init__(self):
    super(Network,self).__init__()
    

    self.fc1 = nn.Linear(in_features=4096, out_features=2048)
    self.fc2 = nn.Linear(in_features=2048, out_features=1024)
    self.out = nn.Linear(in_features=1024, out_features=10)

  def forward(self, t):

    t = t.reshape(-1, 4096)
    t = self.fc1(t)
    t = F.relu(t)
    
    t = self.fc2(t)
    t = F.relu(t)
    
    t = self.out(t)#output shape : (1,10)

    return t

network = Network()

In [91]:
network.eval()
network.to(device)

Network(
  (fc1): Linear(in_features=4096, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=1024, bias=True)
  (out): Linear(in_features=1024, out_features=10, bias=True)
)

In [92]:
optimizer = optim.Adam(network.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

In [93]:
for epoch in range(8):

  _correct = 0
  _loss = 0
  for batch, img in zip(test_loader, pred_output): 
    imgs, labels = batch 
    img = img.to(device)
    labels = labels.to(device)
    preds = network(img) 
    loss = F.cross_entropy(preds, labels) 

    optimizer.zero_grad()
    loss.backward() 
    optimizer.step() 

    _loss += loss.item()
    _correct += preds.argmax(dim=1).eq(labels).sum().item()
    
  print('epoch:', epoch, "correct-",_correct, "loss-",_loss)



epoch: 0 correct- 2253 loss- 19693.97031596887
epoch: 1 correct- 2492 loss- 18895.16367362658
epoch: 2 correct- 2758 loss- 17349.195928880203
epoch: 3 correct- 2903 loss- 17240.380073698878
epoch: 4 correct- 2998 loss- 16544.073643989483
epoch: 5 correct- 3102 loss- 16467.117805796235
epoch: 6 correct- 3237 loss- 16042.969144396191
epoch: 7 correct- 3208 loss- 16044.035657142866


In [94]:
@torch.no_grad()
def get_prediction(model, output):
  all_predictions = torch.tensor([])
  all_predictions = all_predictions.to(device)
  for img in output:
    img = img.to(device)
    predictions = model(img)
    all_predictions = torch.cat((all_predictions, predictions) ,dim=0)

  return all_predictions

In [95]:
testing_predictions = get_prediction(network, pred_output)
actual_lbls = torch.Tensor(testing_data.targets)
actual_lbls = actual_lbls.to(device)
preds_correct =testing_predictions.argmax(dim=1).eq(actual_lbls).sum().item()

print('Correct-', preds_correct)
print('Accuracy-', preds_correct / len(testing_data))

Correct- 3230
Accuracy- 0.323
